In [ ]:
from typing import Tuple, List

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
# possible directions are north, south, west, east - no diagonal
# | north south
# - west east
# L north east
# J north west
# 7 west south
# F east south

# axis
# y is north south with +1 being one step south
# x is west east with +1 being one step east

In [ ]:
# per tile, movement to enter and resulting movement to exit
move_dict = {
    "|": {(1, 0): (1, 0), (-1, 0): (-1, 0)},
    "-": {(0, 1): (0, 1), (0, -1): (0, -1)},
    "L": {(1, 0): (0, 1), (0, -1): (-1, 0)},
    "J": {(1, 0): (0, -1), (0, 1): (-1, 0)},
    "7": {(-1, 0): (0, -1), (0, 1): (1, 0)},
    "F": {(-1, 0): (0, 1), (0, -1): (1, 0)},
}

In [ ]:
def move(start: Tuple[int, int], movement: Tuple[int, int]) -> Tuple[int, int]:
    return (start[0] + movement[0], start[1] + movement[1])


def get_tile(pos: Tuple[int, int]) -> str:
    return lines[pos[0]][pos[1]]

part 1

In [ ]:
# find start position first
start_position = (-1, -1)
for i in range(0, len(lines)):
    for j in range(0, len(lines[i])):
        if lines[i][j] == "S":
            start_position = (i, j)
start_position

In [ ]:
# find start direction by checking allowed inbound movement of next field and if it matches the movement to get there
# because of single loop, there can only be 2
# take first that is found
possible_movements = [(1, 0), (-1, 0), (0, 1), (0, -1)]
first_move = (0, 0)
for pm in possible_movements:
    next_possible_pos = move(start_position, pm)
    if pm in move_dict[get_tile(next_possible_pos)]:
        first_move = pm
        break
first_move

In [ ]:
def move_through_loop(start: Tuple[int, int], first_move: Tuple[int, int]):
    # always move once to avoid finishing immediatly
    steps = 1
    curr_pos = start
    curr_move = first_move

    finished = False

    while not finished:
        curr_pos = move(curr_pos, curr_move)
        # print(f"moved to {curr_pos}")
        curr_tile = get_tile(curr_pos)
        if curr_tile == "S":
            return steps
        # print(curr_tile)
        curr_move = move_dict[curr_tile][curr_move]
        # print(f"next move is {curr_move}")
        steps += 1

    return steps


int(move_through_loop(start_position, first_move) / 2)

part 2

In [ ]:
# that computer graphics lecture many years ago is finally useful
# idea
# Treat loop as polygon, fire a beam and count how often it intersects
# if that is an uneven amount of times, point is inside
# Described in more detail here
# https://en.wikipedia.org/wiki/Point_in_polygon
# Because a lot of vertices are hit, apply that trick to check if next point on path is below
# Thats pretty easy, because that only happens in case of it being F or 7

In [ ]:
def move_through_loop_path(
    start: Tuple[int, int], first_move: Tuple[int, int]
) -> List[Tuple[int, int]]:
    # always move once to avoid finishing immediatly
    path = [start]
    curr_pos = start
    curr_move = first_move

    finished = False

    while not finished:
        curr_pos = move(curr_pos, curr_move)
        # print(f"moved to {curr_pos}")
        curr_tile = get_tile(curr_pos)
        if curr_tile == "S":
            return path
        # print(curr_tile)
        curr_move = move_dict[curr_tile][curr_move]
        # print(f"next move is {curr_move}")
        path.append(curr_pos)

    return path


loop_path = move_through_loop_path(start_position, first_move)

In [ ]:
def shoot_beams(
    tile: Tuple[int, int], loop_tiles: set[Tuple[int, int]], maze: List[str]
):
    if tile in loop_tiles:
        return 0

    # S can be every other symbol, instead of replacing it with the right one,
    # just shoot into the opposite direction if it is hit
    found_start = False

    # shoot east
    s_pos = tile
    n_intersects = 0
    while s_pos[1] < len(maze[0]) - 1:
        s_pos = move(s_pos, (0, 1))
        s_tile = get_tile(s_pos)
        if s_tile == "S":
            found_start = True
            break
        if s_pos in loop_tiles and s_tile in "|F7":
            n_intersects += 1

    if found_start:
        s_pos = tile
        n_intersects = 0
        while s_pos[1] > 0:
            s_pos = move(s_pos, (0, -1))
            if s_pos in loop_tiles and get_tile(s_pos) in "|F7":
                n_intersects += 1

    return n_intersects


loop_tiles = set(loop_path)
shoot_beams((5, 5), loop_tiles, lines)

In [ ]:
inside_fields = 0
for i in range(0, len(lines)):
    for j in range(0, len(lines[0])):
        if (i, j) in loop_tiles:
            continue
        intersects = shoot_beams((i, j), loop_tiles, lines)
        if intersects % 2 == 1:
            inside_fields += 1
inside_fields